In [8]:
import sys
import os
import math
from pathlib import Path

sys.path.insert(0,os.getcwd()+"/..")
from dataset.WeatherBenchData import WeatherBenchData

from torch.utils.data import DataLoader,BatchSampler,SequentialSampler,RandomSampler
import torch
import torch.nn.functional as F
from torch.nn import KLDivLoss, MSELoss
import torch.nn as nn

from diffusers import DDPMScheduler
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import DDPMPipeline
from diffusers import UNet3DConditionModel, UNet2DModel, AutoencoderKL, VQModel,UNet2DConditionModel
from diffusers.models.embeddings import Timesteps,TimestepEmbedding 
from diffusers.models.unet_2d_condition import UNet2DConditionOutput


# Imports for discriminator model 
from taming.modules.discriminator.model import NLayerDiscriminator, weights_init
from taming.modules.losses.vqperceptual import hinge_d_loss, vanilla_d_loss

from collections import namedtuple

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tqdm
import re

from ldm.models.diffusion.ddpm_weather import DDPM
from ldm.models.diffusion.ddim import DDIMSampler

from ldm.modules.diffusionmodules.openaimodel import UNetModel
import properscoring as ps

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
from ldm.modules.diffusionmodules.util import make_ddim_sampling_parameters, make_ddim_timesteps, noise_like
from typing import Any, Dict, List, Optional, Tuple, Union
import torch
from diffusers import DiffusionPipeline,HeunDiscreteScheduler, DDIMScheduler,UniPCMultistepScheduler,DPMSolverMultistepScheduler



# Load the dataset

In [3]:
base_path = "/home/sarkar/Documents/WeatherBench_12"

train_data = WeatherBenchData(Path(base_path),"train",subset=2013)
val_data = WeatherBenchData(Path(base_path),"val",subset=2013)

test_data = WeatherBenchData(Path(base_path),"test")
print(len(train_data),len(test_data))

Total no of channels: 14
<xarray.Dataset>
Dimensions:  (lat: 32, level: 13, lon: 64, time: 350633)
Coordinates:
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * level    (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * time     (time) datetime64[ns] 1979-01-01T07:00:00 ... 2018-12-31T23:00:00
Data variables: (12/14)
    v        (time, level, lat, lon) float32 dask.array<chunksize=(5, 13, 32, 64), meta=np.ndarray>
    t2m      (time, lat, lon) float32 dask.array<chunksize=(5, 32, 64), meta=np.ndarray>
    v10      (time, lat, lon) float32 dask.array<chunksize=(5, 32, 64), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(5, 13, 32, 64), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(5, 13, 32, 64), meta=np.ndarray>
    vo       (time, level, lat, lon) float32 dask.array<chunksize=(5, 13, 

In [4]:
batch_size = 32
train_dataloader = DataLoader(train_data, sampler=BatchSampler(
        SequentialSampler(train_data), batch_size=batch_size, drop_last=False
    ))

test_dataloader = DataLoader(test_data, sampler=BatchSampler(
        SequentialSampler(test_data), batch_size=batch_size, drop_last=False
    ))

val_dataloader = DataLoader(val_data, sampler=BatchSampler(
        SequentialSampler(val_data), batch_size=batch_size, drop_last=False
    ))

In [5]:
from model.w_o_time_emb import DDPM

In [6]:
# Load the model
diffusion = DDPM( 
    cond_model_pth="../outputs/checkpoints/ch_110/v3/vqgan_v3_110ch_ep100.pth",
    #first_stage_model_pth="../outputs/checkpoints/ch_1/v6/vqgan_v6_1ch_ep200.pth"
    first_stage_model_pth="../outputs/checkpoints/ch_1/v11/vqgan_v11_1ch_best_ep99.pth",
    device="cuda"
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 # Load the model                                                                             │
│ ❱ 2 diffusion = DDPM(                                                                            │
│   3 │   cond_model_pth="../outputs/checkpoints/ch_110/v3/vqgan_v3_110ch_ep100.pth",              │
│   4 │   #first_stage_model_pth="../outputs/checkpoints/ch_1/v6/vqgan_v6_1ch_ep200.pth"           │
│   5 │   first_stage_model_pth="../outputs/checkpoints/ch_1/v11/vqgan_v11_1ch_best_ep99.pth",     │
│                                                                                                  │
│ /home/sarkar/Documents/code/test/notebooks/../model/w_o_time_emb.py:125 in __init__              │
│                                                                                                  │
│   122 │   │   #self.model = self.model.to("cuda")                                                │
│   123 │   │                                                                                      │
│   124 │   │   # Model that encodes the latents of the 1channel geo-potentials and 110 channel    │
│ ❱ 125 │   │   self.initialise_models()                                                           │
│   126 │   │                                                                                      │
│   127 │   │                                                                                      │
│   128 │   │   self.loss_type = loss_type                                                         │
│                                                                                                  │
│ /home/sarkar/Documents/code/test/notebooks/../model/w_o_time_emb.py:171 in initialise_models     │
│                                                                                                  │
│   168 │   │   self.cond_stage_model = VQModel(in_channels= self.channels, out_channels=self.ch   │
│   169 │   │   self.first_stage_model = VQModel(in_channels= 1, out_channels=1,latent_channels=   │
│   170 │   │   #load weights                                                                      │
│ ❱ 171 │   │   self.first_stage_model.load_state_dict(torch.load(self.first_stage_model_pth)["s   │
│   172 │   │   self.cond_stage_model.load_state_dict(torch.load(self.cond_model_pth)["state_dic   │
│   173 │   │                                                                                      │
│   174 │   │   self.first_stage_model = self.first_stage_model.eval()                             │
│                                                                                                  │
│ /home/sarkar/anaconda3/envs/weather/lib/python3.10/site-packages/torch/serialization.py:791 in   │
│ load                                                                                             │
│                                                                                                  │
│    788 │   if 'encoding' not in pickle_load_args.keys():                                         │
│    789 │   │   pickle_load_args['encoding'] = 'utf-8'                                            │
│    790 │                                                                                         │
│ ❱  791 │   with _open_file_like(f, 'rb') as opened_file:                                         │
│    792 │   │   if _is_zipfile(opened_file):                                                      │
│    793 │   │   │   # The zipfile reader is going to advance the current file position.           │
│    794 │   │   │   # If we want to actually tail call to torch.jit.load, we need to              │
│                                                            

In [ ]:
import torch
from diffusers import DiffusionPipeline,HeunDiscreteScheduler, DDIMScheduler,UniPCMultistepScheduler,DPMSolverMultistepScheduler


class MyPipeline(DiffusionPipeline):
    def __init__(self, model, scheduler):
        super().__init__()

        self.register_modules(model=model, scheduler=scheduler)

    @torch.no_grad()
    def __call__(self,day,hour, cond, shape, batch_size: int = 1, num_inference_steps: int = 50):
        # Sample gaussian noise to begin loop
        c,h,w = shape
        image = torch.randn((batch_size, c,h,w ))

        image = image.to(self.device)
        c = self.model.cond_stage_model.encode(cond).latents
        c = c.view(cond.size(0), 64, -1)
        cc = torch.broadcast_to(c, (batch_size, 64, 32*64))
        # set step values
        self.scheduler.set_timesteps(num_inference_steps)

        for t in self.progress_bar(self.scheduler.timesteps):
            # 1. predict noise model_output
            image = self.scheduler.scale_model_input(image,t)
            model_output = self.model(image,t,day,hour,cc)

            # 2. predict previous mean of image x_t-1 and add variance depending on eta
            # eta corresponds to η in paper and should be between [0, 1]
            # do x_t -> x_t-1
            image = self.scheduler.step(model_output, t, image).prev_sample

        #image = (image / 2 + 0.5).clamp(0, 1)
        #image = image.cpu().permute(0, 2, 3, 1).numpy()
        image = diffusion.first_stage_model.decode(image)
        return image
    
    

In [ ]:
# The solver can be chaanged to UHeunDiscreteScheduler, DDIMScheduler,UniPCMultistepScheduler for other solver methds
scheduler = DPMSolverMultistepScheduler(trained_betas=diffusion.betas.cpu(),use_karras_sigmas=True)
diffusion = diffusion.to("cuda")
pipe = MyPipeline(diffusion,scheduler)

In [ ]:
# Run solver
no_ensemble_members = 50
ensemble_members = []
total_crps =0
s = 40

batch_size = 128*2
test_dataloader = DataLoader(test_data, sampler=BatchSampler(
        SequentialSampler(test_data), batch_size=batch_size, drop_last=False
    ))

for i, item in enumerate(test_dataloader):
    cond_,tgt_ = item

    hour = tgt_["hour"].squeeze(0)* 2 * math.pi / 24
    day = (tgt_["day"].squeeze(0) - 1 )* 2 * math.pi / 365

    tgt = tgt_["data"]
    cond = cond_["data"]
    tgt = tgt.reshape(tgt.shape[1], 1,  32,64)
    cond = cond.squeeze(0)
    
    cond = cond.to("cuda")
    day = day.to("cuda")
    hour = hour.to("cuda")

    t1 = datetime.now()
    for i in range(no_ensemble_members):
        op = pipe(day,hour,cond,[4, 16, 32], num_inference_steps=s,batch_size=cond.shape[0])
        forecasts =  op[0][0].cpu().detach().numpy()
        ensemble_members.append(forecasts)
        
    t2 = datetime.now()
    break

print(t2-t1)
no_ensemble_members = 50
ensemble_members = []
total_crps =0
s = 40

batch_size = 128*2
test_dataloader = DataLoader(test_data, sampler=BatchSampler(
        SequentialSampler(test_data), batch_size=batch_size, drop_last=False
    ))
all_ens = []
for i, item in enumerate(test_dataloader):
    cond_,tgt_ = item

    hour = tgt_["hour"].squeeze(0)* 2 * math.pi / 24
    day = (tgt_["day"].squeeze(0) - 1 )* 2 * math.pi / 365

    tgt = tgt_["data"]
    cond = cond_["data"]
    tgt = tgt.reshape(tgt.shape[1], 1,  32,64)
    cond = cond.squeeze(0)
    
    cond = cond.to("cuda")
    day = day.to("cuda")
    hour = hour.to("cuda")

    t1 = datetime.now()
    ensemble_members = []
    for i in range(no_ensemble_members):
        op = pipe(day,hour,cond,[4, 16, 32], num_inference_steps=s,batch_size=cond.shape[0])
        forecasts =  op[0][0].cpu().detach().numpy()
        ensemble_members.append(forecasts)
        
    t2 = datetime.now()
    
    stacked_ensemble = np.stack(ensemble_members)
    all_ens.append(stacked_ensemble)
stacked_ensemble = np.vstack(all_ens)
np.save("karras_time_emb_40_ep200.npy",stacked_ensemble)

In [ ]:
stacked_ensemble = np.stack(ensemble_members)
total_crps = 0
total_rmse = 0
weights_lat = np.array([0.07704437, 0.23039114, 0.38151911, 0.52897285, 0.67133229,
       0.80722643, 0.93534654, 1.05445875, 1.16341595, 1.26116882,
       1.34677594, 1.41941287, 1.47838008, 1.52310968, 1.55317091,
       1.56827425, 1.56827425, 1.55317091, 1.52310968, 1.47838008,
       1.41941287, 1.34677594, 1.26116882, 1.16341595, 1.05445875,
       0.93534654, 0.80722643, 0.67133229, 0.52897285, 0.38151911,
       0.23039114, 0.07704437])
weights_lat = weights_lat[:, np.newaxis]

for i in range(stacked_ensemble.shape[1]):
    forecast = stacked_ensemble[:,i,:,:].squeeze() * 59563.01 + 77907.086
    observations = tgt[i].squeeze() * 59563.01 + 77907.086
    
    crps_score = ps.crps_ensemble(observations,forecast,axis=0)
    
    ensemble_mean = np.mean(forecast,axis = 0)
    rmse = np.sqrt(np.mean(  weights_lat * (ensemble_mean - observations.cpu().detach().numpy()) ** 2) )

    print(np.mean(weights_lat * crps_score),rmse)
    total_crps += np.mean(weights_lat * crps_score)
    total_rmse += rmse

print("RMSE", total_rmse/stacked_ensemble.shape[1])
print("CRPS", total_crps/stacked_ensemble.shape[1])
print("time for each sample ensemble",(t2-t1)/batch_size)